In [1]:
import pandas as pd
import numpy as np

import time


In [2]:
import os

path_dataset = '../Dataset'

path_so = os.path.join(path_dataset, 'Stack Overflow')
path_ts = os.path.join(path_dataset, 'Tool-specific Others')
path_labeling = os.path.join(path_dataset, 'Labeling')  

path_so_raw = os.path.join(path_so, 'Raw')
path_ts_raw = os.path.join(path_ts, 'Raw')
path_so_filtered = os.path.join(path_so, 'Filtered')
path_ts_filtered = os.path.join(path_ts, 'Filtered')

if not os.path.exists(path_dataset):
    os.makedirs(path_dataset)

if not os.path.isdir(path_so):
    os.mkdir(path_so)

if not os.path.isdir(path_ts):
    os.mkdir(path_ts)

if not os.path.isdir(path_labeling):
    os.mkdir(path_labeling)

if not os.path.isdir(path_so_raw):
    os.mkdir(path_so_raw)

if not os.path.isdir(path_ts_raw):
    os.mkdir(path_ts_raw)

if not os.path.isdir(path_so_filtered):
    os.mkdir(path_so_filtered)

if not os.path.isdir(path_ts_filtered):
    os.mkdir(path_ts_filtered)


In [4]:
tool2tag = {
    'Amazon SageMaker': {'amazon-sagemaker', 'amazon-sagemaker-experiments', 'amazon-sagemaker-studio'},
    'Azure Machine Learning': {'azure-machine-learning-service', 'azure-machine-learning-studio', 'azure-machine-learning-workbench'},
    'ClearML': {'clearml'},
    'Comet': {'comet-ml'},
    'DVC': {'dvc'},
    'Kedro': {'kedro'},
    'MLflow': {'mlflow'},
    'MLRun': {'mlrun'},
    'Neptune': {'neptune'},
    'Optuna': {'optuna'},
    'Sacred': {'python-sacred'},
    'Vertex AI': {'google-cloud-vertex-ai'},
    'Weights & Biases': {'wandb'}
}

tools_keywords = {
    'Amazon SageMaker': ['amazon sagemaker', 'aws sagemaker', 'sagemaker'],
    'Azure Machine Learning': ['microsoft azure machine learning', 'azure machine learning', 'microsoft azure ml', 'microsoft azureml', 'azure ml', 'azureml'],
    'ClearML': ['clearml'],
    'Comet': ['comet'],
    'Domino': ['domino'],
    'DVC': ['dvc'],
    'Guild AI': ['guild'],
    'Kedro': ['kedro'],
    'MLflow': ['mlflow'],
    'Neptune': ['neptune'],
    'Optuna': ['optuna'],
    'Polyaxon': ['polyaxon'],
    'Sacred': ['sacred'],
    'SigOpt': ['sigopt'],
    'Vertex AI': ['google vertex ai', 'vertex ai'],
    'Weights & Biases': ['weights & biases', 'weights and biases', 'wandb']
}

tool_no_accepted_answer = {
    'Domino', 
    'DVC', 
    'Guild AI"', 
    'MLflow', 
    'Polyaxon', 
    'SigOpt'
}


In [5]:
import openai

openai.api_key = os.getenv('OPENAI_API_KEY')


In [2]:
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = 'all'


In [9]:
import requests
import json

# function to scrape the posts from the tool-specific discussion fora


def scrape_post(base_url, page_suffix, file_name):
    page = -1
    post_list = []

    while True:
        page = page + 1
        page_url = base_url + page_suffix + str(page)
        topic_list = requests.get(page_url).json()['topic_list']

        for topic in topic_list['topics']:
            post_url = base_url + 't/' + \
                topic['slug'] + '/' + str(topic['id'])

            post = {}
            post['Question_title'] = topic['title']
            post['Question_link'] = post_url
            post['Question_created_time'] = topic['created_at']
            post['Question_answer_count'] = topic['posts_count'] - 1
            post['Question_score_count'] = topic['like_count']
            post['Question_view_count'] = topic['views']
            post['Question_has_accepted_answer'] = topic['has_accepted_answer']
            comments = requests.get(
                post_url + '.json').json()['post_stream']['posts']
            post['Question_body'] = comments[0]['cooked']
            
            answer_list = []
            for comment in comments[1:]:
                answer = {}
                answer['Answer_created_time'] = comment['created_at']
                answer['Answer_body'] = comment['cooked']
                answer['Answer_has_accepted'] = comment['accepted_answer']
                answer_list.append(answer)                
            post['Answer_list'] = answer_list
            
            post_list.append(post)
            time.sleep(5)

        if 'more_topics_url' not in topic_list.keys():
            break

    with open(os.path.join(path_ts_raw, file_name), 'w') as outfile:
        json_post_list = json.dumps(post_list, indent='\t')
        outfile.write(json_post_list)


In [ ]:
# scrape posts from Guild AI
base_url = 'https://my.guild.ai/'
page_suffix = 'c/troubleshooting/6.json?page='
file_name = 'Guild AI.json'
scrape_post(base_url, page_suffix, file_name)


In [49]:
# scrape posts from Weights & Biases
base_url = 'https://community.wandb.ai/'
page_suffix = 'c/w-b-support/36.json?page='
file_name = 'Weights & Biases.json'
scrape_post(base_url, page_suffix, file_name)


In [41]:
# scrape posts from SigOpt
base_url = 'https://community.sigopt.com/'
page_suffix = 'c/general-discussion/9.json?page='
file_name = 'SigOpt.json'
scrape_post(base_url, page_suffix, file_name)


In [42]:
# scrape posts from DVC
base_url = 'https://discuss.dvc.org/'
page_suffix = 'c/questions/9.json?page='
file_name = 'DVC.json'
scrape_post(base_url, page_suffix, file_name)


In [24]:
import glob

# exclude tool-specific posts with negative upvote count
df_ts_questions = pd.DataFrame()
total_post = 0

for file_name in glob.glob(os.path.join(path_ts_raw, '*.json')):
    repos = pd.read_json(file_name)
    total_post += len(repos)    
    if 'Question_score_count' in repos.columns:
        repos = repos[repos['Question_score_count'] > -1]
    repos['Tool'] = os.path.split(file_name)[1].split('.')[0]
    df_ts_questions = pd.concat([df_ts_questions, repos], ignore_index=True)

df_ts_questions['Question_comment_count'] = df_ts_questions['Question_comment_count'].fillna(0)
df_ts_questions.to_json(os.path.join(path_ts_filtered,
                                     'questions.json'), orient='records', indent=4)
print(total_post, df_ts_questions.shape[0])

3812
3812


In [ ]:
import google.auth
import pandas_gbq

credentials, _ = google.auth.default()

pandas_gbq.context.credentials = credentials
pandas_gbq.context.project = 'stack-overflow-dataset-330612'

with open(os.path.join(path_so_raw, 'bigquery.sql'), 'r') as sql_file:
    sql = sql_file.read()
    df = pandas_gbq.read_gbq(sql)
    df['Question_tags'] = df['Question_tags'].str.split('|')
    df['Question_favorite_count'] = df['Question_favorite_count'].fillna(0)

len(df)

In [ ]:
# create tag collection
tags = set()
for key, value in tool2tag.items():
    tags = tags.union(value)


In [ ]:
# split tags
df['Question_valid_tags'] = [[] for _ in range(len(df))]
for index, row in df.iterrows():
    df.at[index, 'Question_valid_tags'] = list(
        tags.intersection(set(row['Question_tags'])))


In [ ]:
# count post number with different tags
arity = 0
while True:
    post_number = df[df['Question_valid_tags'].map(len) > arity].shape[0]
    if post_number < 1:
        break
    arity = arity + 1
    print(f'Posts with at least {arity} tags has {post_number} in total.')


Posts with at least 1 tags has 5308 in total.
Posts with at least 2 tags has 220 in total.
Posts with at least 3 tags has 18 in total.


In [ ]:
# exclude Stack Overflow posts with unrelated tags
df_valid = df[df['Question_valid_tags'].map(len) > 0]
df_valid['Question_link'] = df_valid['Question_id'].apply(
    lambda x: f'https://stackoverflow.com/questions/{x}')
len(df_valid)

C:\Users\zhimi\AppData\Local\Temp\ipykernel_8032\3717773066.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_valid['Question_link'] = df_valid['Question_id'].apply(


5308

In [ ]:
# exclude Stack Overflow posts with negative upvote count
df_qualified = df_valid[df_valid['Question_score_count'] > -1]
len(df_qualified)

5175

In [ ]:
# create a map from tag to tool
tag2tool = dict()
for key, value in tool2tag.items():
    for elem in value:
        tag2tool.setdefault(elem, key)
        

In [ ]:
# extract Stack Overflow post collection with multiple tags based on the tool map
for index, row in df_qualified.iterrows():
    tags = set()
    for tag in row['Question_valid_tags']:
        tags.add(tag2tool[tag])
    df_qualified.at[index, 'Question_valid_tags'] = sorted(list(tags))
    

In [ ]:
# check how the posts with more than one tags look like
df_multiply_tagged = df_qualified[df_qualified['Question_valid_tags'].map(
    len) > 1]
df_multiply_tagged['Question_valid_tags'].value_counts()


[Amazon SageMaker, MLflow]                 16
[Azure Machine Learning, MLflow]           11
[Kedro, MLflow]                             4
[Azure Machine Learning, Kedro, MLflow]     2
[DVC, MLflow]                               1
[Kedro, Neptune]                            1
[MLflow, Sacred]                            1
Name: Question_valid_tags, dtype: int64

In [43]:
# create Stack Overflow post collection with exclusive tags
multiply_tagged_posts_split = []
df_qualified.assign(Tool='')

for index, row in df_qualified.iterrows():
    tags = row['Question_valid_tags']
    df_qualified.at[index, 'Tool'] = tags[0]
    if len(tags) > 1:
        for tag in tags[1:]:
            series = row.copy()
            series['Tool'] = tag
            multiply_tagged_posts_split.append(series)

df_multiply_tagged_posts_split = pd.DataFrame(multiply_tagged_posts_split)
df_qualified_exclusive_tagged = pd.concat(
    [df_qualified, df_multiply_tagged_posts_split], ignore_index=True)

df_qualified_exclusive_tagged['Challenge_self_resolution'] = df_qualified_exclusive_tagged['Poster_id'] == df_qualified_exclusive_tagged['Answerer_id']

del df_qualified_exclusive_tagged['Poster_id']
del df_qualified_exclusive_tagged['Answerer_id']
del df_qualified_exclusive_tagged['Question_id']
# remove null age due to anonymity
del df_qualified_exclusive_tagged['Poster_age']
del df_qualified_exclusive_tagged['Answerer_age']
del df_qualified_exclusive_tagged['Question_valid_tags']

df_qualified_exclusive_tagged.to_json(os.path.join(
    path_so_filtered, 'questions.json'), indent=4, orient='records')


In [5]:
# create question dataset

df_question_so = pd.read_json(os.path.join(path_so_filtered, 'questions.json'))
df_question_ts = pd.read_json(os.path.join(path_ts_filtered, 'questions.json'))

df_question_so['Platform'] = 'Stack Overflow'
df_question_so['Question_closed_time'] = df_question_so['Answer_created_time']

df_question_ts['Platform'] = 'Tool-specific'
df_question_ts['Question_closed_time'] = np.nan

# Retrieve the creation time of the acctepted answer 
for index, row in df_question_ts.iterrows():
    if row['Tool'] in tool_no_accepted_answer or not row['Question_has_accepted_answer']:
        continue
    for comment in row['Answer_list']:
        if comment['Answer_has_accepted']:
            df_question_ts.at[index, 'Answer_body'] = comment['Answer_body']
            df_question_ts.at[index, 'Question_closed_time'] = pd.to_datetime(comment['Answer_created_time'])
            if 'Answer_comment_count' in comment:
                df_question_ts.at[index, 'Answer_comment_count'] = pd.to_datetime(comment['Answer_comment_count'])
            if 'Answer_score_count' in comment:
                df_question_ts.at[index, 'Answer_score_count'] = pd.to_datetime(comment['Answer_score_count'])
            break

df_questions = pd.concat([df_question_so, df_question_ts], ignore_index=True)

del df_questions['Question_tags']
del df_questions['Question_has_accepted_answer']
del df_questions['Answer_list']
del df_questions['Answer_created_time']

df_questions.to_json(os.path.join(
    path_labeling, 'original.json'), indent=4, orient='records')


In [20]:
# add potential field to questions for later filling
df_questions = pd.read_json(os.path.join(path_labeling, 'original.json'))

df_questions['Question_original_content'] = np.nan
df_questions['Question_preprocessed_content'] = np.nan
df_questions['Question_gpt_summary_original'] = np.nan
df_questions['Question_gpt_summary'] = np.nan
df_questions['Answer_original_content'] = np.nan
df_questions['Answer_preprocessed_content'] = np.nan
df_questions['Answer_gpt_summary_original'] = np.nan
df_questions['Answer_gpt_summary'] = np.nan

df_questions.to_json(os.path.join(path_labeling, 'questions.json'), indent=4, orient='records')


In [4]:
# content filtering patterns

from gensim.parsing.preprocessing import preprocess_string
from string import ascii_lowercase
from bs4 import BeautifulSoup
import re

regex = r"(ftp|https?://[^\s]+)|(<.*?>)|({.*?})|((!)?\[.*?\])|(\(.*?\))|(\`{3}.+?\`{3})|(\`{2}.+?\`{2})|(\`{1}.+?\`{1})|([^\s]*[<=>]=[^\s]+)|(@[^\s]+)|([^\s]*\\[^\s]+)|([^\s]+\/[^\s]+)|([^\s]+\.[^\s]+)|([^\s]+-[^\s]+)|([^\s]+_[^\s]+)|(_+[^\s]+_*)|(_*[^\s]+_+)|([0-9\|\-\r\n\t\"\-#*=~:{}\(\)\[\]<>]+)"

len_max = len('aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa')

def preprocess_text(text, remove_code=False):          
    soup = BeautifulSoup(text, 'html.parser')
    remove_tags = ['script', 'style']
    remove_tags.append('code') if remove_code else None
    for tag in soup(remove_tags):
        tag.decompose()
    text = soup.get_text()
    
    text = text.lower().encode('ascii', errors='ignore').decode('ascii')
    for tool_keywords in tools_keywords.values():
        for tool_keyword in tool_keywords:
            if tool_keyword in text:
                text = text.replace(tool_keyword, '')
    
    text = re.sub(regex, ' ', text, 0, re.DOTALL) if remove_code else text
    
    # remove repeated letters
    for time in range(3, len_max + 1):
        for letter in ascii_lowercase:
            text = text.replace(letter * time, '')
    
    text = preprocess_string(text)
    text = ' '.join(text)
    return text


In [ ]:
# create prompt for gpt model

import random

prompt_question = 'Your task is to provide a brief and accurate summary of the challenges that the user has encountered based on the given text. Your summary should be concise, highlighting only the most important details related to the challenges faced by the user. Please note that your response should focus on providing an objective and factual summary of the challenges without including any personal opinions or biases.\n###'
prompt_answer = 'Given a challenge-discussion pair, please extract any possible solutions mentioned in the discussion and provide a brief and accurate summary of them. If no solution is mentioned, please indicate that there are no solutions provided. Please note that your response should focus on providing an objective and factual summary without including any personal opinions or biases.\n###'

def retry_with_backoff(fn, retries=2, backoff_in_seconds=1, *args, **kwargs):
    x = 0

    if args is None:
        args = []
    if kwargs is None:
        kwargs = {}

    while True:
        try:
            return fn(*args, **kwargs)
        except:
            if x == retries:
                raise

            sleep = backoff_in_seconds * 2 ** x + random.uniform(0, 1)
            time.sleep(sleep)
            x += 1
            

In [6]:
# Experiment 1

df_questions = pd.read_json(os.path.join(
    path_labeling, 'questions.json'))

for index, row in df_questions.iterrows():
    question = preprocess_text(row['Question_title']) + ' ' + preprocess_text(str(row['Question_body']))

    if len(question.split()) < 6:
        df_questions.drop(index, inplace=True)
        print(question)
    else:
        df_questions.at[index, 'Question_original_content'] = question

df_questions.to_json(os.path.join(path_labeling,
                     'questions.json'), indent=4, orient='records')


/Users/jimmy/anaconda3/lib/python3.10/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(
/Users/jimmy/anaconda3/lib/python3.10/site-packages/bs4/__init__.py:404: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  warnings.warn(


In [7]:
df_questions = pd.read_json(os.path.join(path_labeling, 'questions.json'))
df_questions.groupby('Platform').count()['Question_title'].reset_index()

,Platform,Question_title
0,Stack Overflow,5213
1,Tool-specific,3788


In [8]:
df_questions = pd.read_json(os.path.join(path_labeling, 'questions.json'))
df_questions.groupby('Tool').count()['Question_title'].reset_index()

,Tool,Question_title
0,Amazon SageMaker,2776
1,Azure Machine Learning,2974
2,ClearML,40
3,Comet,10
4,DVC,439
5,Domino,13
6,Guild AI,118
7,Kedro,149
8,MLflow,831
9,Neptune,8


In [8]:
# Experiment 2

df_questions = pd.read_json(os.path.join(
    path_labeling, 'questions.json'))

for index, row in df_questions.iterrows():
    if index % 50 == 49:
        print(f'persisting on question {index}')
        df_questions.to_json(os.path.join(
            path_labeling, 'questions.json'), indent=4, orient='records')
        
    if row['Question_gpt_summary_original']:
        continue

    try:
        prompt = prompt_question + 'Title: ' + row['Question_title'] + ' Body: ' + row['Question_body'] + '###\n'
        response = retry_with_backoff(
            openai.ChatCompletion.create,
            model='gpt-4-32k',
            messages=[
                {"role": "system", "content": "You are an accurate summarizer."},
                {"role": "user", "content": prompt},
            ],
            temperature=0,
            max_tokens=150,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0,
            timeout=10,
            stream=False
        )
        content = response['choices'][0]['message']['content'].strip()
        df_questions.at[index, 'Question_gpt_summary_original'] = content
        df_questions.at[index, 'Question_gpt_summary'] = preprocess_text(content)
    except Exception as e:
        # output unsuccesful requests
        print(f'{e} on question {row["Question_link"]}')

    time.sleep(1)

df_questions.to_json(os.path.join(
    path_labeling, 'questions.json'), indent=4, orient='records')


persisting on question 49
persisting on question 99
persisting on question 149
persisting on question 199
persisting on question 249
persisting on question 299
persisting on question 349
persisting on question 399
persisting on question 449
persisting on question 499
persisting on question 549
persisting on question 599
persisting on question 649
persisting on question 699
persisting on question 749
persisting on question 799
persisting on question 849
persisting on question 899
persisting on question 949
persisting on question 999
persisting on question 1049
persisting on question 1099
persisting on question 1149
persisting on question 1199
persisting on question 1249
persisting on question 1299
persisting on question 1349
persisting on question 1399
persisting on question 1449
persisting on question 1499
persisting on question 1549
persisting on question 1599
persisting on question 1649
persisting on question 1699
persisting on question 1749
persisting on question 1799
persisting on 

In [10]:
assert df_questions.shape[0] == df_questions[df_questions['Question_gpt_summary_original'].str.len() > 0].shape[0]


In [11]:
# Experiment 3

df_questions = pd.read_json(os.path.join(
    path_labeling, 'questions.json'))

for index, row in df_questions.iterrows():
    content = preprocess_text(row['Question_title'], remove_code=True) + ' ' + preprocess_text(str(row['Question_body']), remove_code=True)
    df_questions.at[index, 'Question_preprocessed_content'] = content

df_questions.to_json(os.path.join(
    path_labeling, 'questions.json'), indent=4, orient='records')


/Users/jimmy/anaconda3/lib/python3.10/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(
/Users/jimmy/anaconda3/lib/python3.10/site-packages/bs4/__init__.py:404: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  warnings.warn(


In [12]:
# Experiment 4

df_questions = pd.read_json(os.path.join(path_labeling, 'questions.json'))

for index, row in df_questions.iterrows():
    if pd.notna(row['Answer_body']):
        df_questions.at[index, 'Answer_original_content'] = preprocess_text(row['Answer_body'])

df_questions.to_json(os.path.join(path_labeling, 'questions.json'), indent=4, orient='records')


/Users/jimmy/anaconda3/lib/python3.10/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [9]:
# Experiment 5

df_questions = pd.read_json(os.path.join(
    path_labeling, 'questions.json'))

for index, row in df_questions.iterrows():
    if index % 50 == 49:
        print(f'persisting on question {index}')
        df_questions.to_json(os.path.join(
            path_labeling, 'questions.json'), indent=4, orient='records')
    
    if pd.isna(row['Answer_body']) or row['Answer_gpt_summary_original']:
        continue

    try:
        prompt = prompt_answer + 'Challenge: ' + row['Question_gpt_summary_original'] + ' Discussion: ' + row['Answer_body'] + '###\n'
        response = retry_with_backoff(
            openai.ChatCompletion.create,
            model='gpt-4-32k',
            messages=[
                {"role": "system", "content": "You are an accurate summarizer."},
                {"role": "user", "content": prompt},
            ],
            temperature=0,
            max_tokens=150,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0,
            timeout=10,
            stream=False
        )
        content = response['choices'][0]['message']['content'].strip()
        df_questions.at[index, 'Answer_gpt_summary_original'] = content
        df_questions.at[index, 'Answer_gpt_summary'] = preprocess_text(content)
    except Exception as e:
        # output unsuccesful requests
        print(f'{e} on question {row["Question_link"]}')

    time.sleep(1)

df_questions.to_json(os.path.join(
    path_labeling, 'questions.json'), indent=4, orient='records')


persisting on question 49
persisting on question 99
persisting on question 149
persisting on question 199
persisting on question 249
persisting on question 299
persisting on question 349
persisting on question 399
persisting on question 449
persisting on question 499
persisting on question 549
persisting on question 599
persisting on question 649
persisting on question 699
persisting on question 749
persisting on question 799
persisting on question 849
persisting on question 899
persisting on question 949
persisting on question 999
persisting on question 1049
persisting on question 1099
persisting on question 1149
persisting on question 1199
persisting on question 1249
persisting on question 1299
persisting on question 1349
persisting on question 1399
persisting on question 1449
persisting on question 1499
persisting on question 1549
persisting on question 1599
persisting on question 1649
persisting on question 1699
persisting on question 1749
persisting on question 1799
persisting on 

In [13]:
# Experiment 6

df_questions = pd.read_json(os.path.join(path_labeling, 'questions.json'))

for index, row in df_questions.iterrows():
    if pd.notna(row['Answer_body']):
        df_questions.at[index, 'Answer_preprocessed_content'] = preprocess_text(row['Answer_body'], remove_code=True)

df_questions.to_json(os.path.join(path_labeling, 'questions.json'), indent=4, orient='records')


/Users/jimmy/anaconda3/lib/python3.10/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [17]:
df_questions = pd.read_json(os.path.join(path_labeling, 'questions.json'))

# output the number of asset-management-related Q&A questions
len(df_questions)


9001

In [7]:
# sample size is based on the recommendation from https://www.calculator.net/sample-size-calculator.html

sample_size = 369

df_questions = pd.read_json(os.path.join(
    path_labeling, 'questions.json'))

df_sample = df_questions[~df_questions['Answer_body'].isna()].sample(n=sample_size, random_state=42)

df_sample.to_json(os.path.join(
    path_labeling, 'sample.json'), indent=4, orient='records')
